In [ ]:
import math, time
from typing import Any, Dict, Callable
import json


# Tools (functions the agent can call)

def get_weather(city: str) -> str:
    """Mock weather API: return a fake but plausible weather string."""
    data = {
        'Lahore': 'Sunny, 33°C',
        'Karachi': 'Hot, 35°C',
        'Islamabad': 'Pleasant, 28°C'
    }
    return data.get(city, f'No weather data for {city}.')

def calculator(expr: str) -> str:
    """Evaluate a math expression safely (limited eval)."""
    try:
        allowed = {"__builtins__": {}}
        allowed.update({k: getattr(math, k) for k in dir(math) if not k.startswith('_')})
        result = eval(expr, allowed, {})
        return str(result)
    except Exception as e:
        return f'Calculator error: {e}'

def get_stock_price(ticker: str) -> str:
    """Mock stock lookup."""
    sample = {
        'AAPL': 'AAPL: $175.32',
        'TSLA': 'TSLA: $430.12',
        'GOOG': 'GOOG: $135.44'
    }
    return sample.get(ticker.upper(), f'No data for {ticker}')

SIMPLE_KB = {
    'react agent': 'ReAct is a pattern combining reasoning and tool usage.',
    'function calling': 'Function calling means an LLM returns a structured call to an external tool.'
}
def kb_search(query: str) -> str:
    q = query.lower()
    matches = [v for k,v in SIMPLE_KB.items() if k in q or q in k]
    return '\n'.join(matches) if matches else f'No KB hits for "{query}".'

TOOLS: Dict[str, Callable[[str], str]] = {
    'get_weather': get_weather,
    'calculator': calculator,
    'get_stock_price': get_stock_price,
    'kb_search': kb_search
}


In [ ]:
def simulated_llm_for_function_call(prompt: str) -> Any:
    """A tiny rule-based simulated LLM that sometimes returns function_call instructions.
    The return format:
      - {'type': 'answer', 'content': '...'}
      - {'type': 'function_call', 'name': 'tool_name', 'arguments': {...}}
    """
    p = prompt.lower()
    # If user asks for weather, request function call
    if 'weather' in p:
        # extract a city name naive
        for city in ['Lahore', 'Karachi', 'Islamabad']:
            if city.lower() in p:
                return {'type': 'function_call', 'name': 'get_weather', 'arguments': {'city': city}}
        # if no known city, ask for city (here we simulate calling with 'Lahore')
        return {'type': 'function_call', 'name': 'get_weather', 'arguments': {'city': 'Lahore'}}

    # If user asks a math expression
    if any(ch in p for ch in ['+', '-', '*', '/', 'sqrt']):
        import re
        m = re.search(r"([\d\.\s\+\-\*\/\(\)]+)", prompt)
        expr = m.group(1).strip() if m else '2+2'
        return {'type': 'function_call', 'name': 'calculator', 'arguments': {'expr': expr}}

    # If user asks about a stock
    if 'stock' in p or 'price of' in p:
        # naive ticker detection
        for ticker in ['AAPL','TSLA','GOOG']:
            if ticker.lower() in p:
                return {'type': 'function_call', 'name': 'get_stock_price', 'arguments': {'ticker': ticker}}
        # fallback: ask for AAPL
        return {'type': 'function_call', 'name': 'get_stock_price', 'arguments': {'ticker': 'AAPL'}}

    # If user asks for a definition, call kb_search
    if 'what is' in p or 'define' in p:
        # call kb_search
        q = prompt.split('what is')[-1].strip() if 'what is' in p else prompt
        return {'type': 'function_call', 'name': 'kb_search', 'arguments': {'query': q}}

    # default: return a direct answer
    return {'type': 'answer', 'content': "I'm a simulated LLM. I can call functions for you."}


In [ ]:
def run_function_calling_agent(question: str, max_rounds: int = 3, verbose: bool = True) -> Dict[str, Any]:
    """Run a simple function-calling agent using the simulated LLM.
    Returns a dict with final answer and call history.
    """
    history = []  # tool call history
    # initial prompt
    prompt = question
    for round_idx in range(1, max_rounds + 1):
        if verbose:
            print(f"\n=== Round {round_idx} ===")
            print("Prompt to LLM:\n", prompt)
        llm_response = simulated_llm_for_function_call(prompt)
        if verbose:
            print("LLM response:\n", llm_response)

        if llm_response['type'] == 'answer':
            # final answer
            return {'answer': llm_response['content'], 'history': history}

        if llm_response['type'] == 'function_call':
            fname = llm_response['name']
            args = llm_response.get('arguments', {})
            func = TOOLS.get(fname)
            if not func:
                observation = f"Error: unknown tool '{fname}'"
            else:
                # map argument names to function parameter names
                # each tool in this demo expects a single positional string arg except calculator
                if fname == 'calculator':
                    observation = func(args.get('expr', ''))
                elif fname == 'get_weather':
                    observation = func(args.get('city', ''))
                elif fname == 'get_stock_price':
                    observation = func(args.get('ticker', ''))
                elif fname == 'kb_search':
                    observation = func(args.get('query', ''))
                else:
                    observation = 'No handler for this tool in demo.'

            history.append({'call': fname, 'arguments': args, 'observation': observation})
            if verbose:
                print('Executed tool -> observation:\n', observation)

            # Prepare the next prompt: include the observation so the LLM can reason with it.
            prompt = question + '\n' + '\n'.join([f"CALL: {h['call']} ARGS: {h['arguments']} => {h['observation']}" for h in history])
            # continue to next round
            continue

        # fallback
        return {'answer': 'LLM returned unrecognized response', 'history': history}

    return {'answer': 'Max rounds reached without final answer', 'history': history}


In [ ]:
examples = [
    'What is the weather in Lahore today?',
    'Compute 12 * (3 + 4)',
    'What is the price of TSLA stock?',
    'Define function calling in one sentence'
]
for q in examples:
    print('\n==============================')
    print('User question:', q)
    out = run_function_calling_agent(q, verbose=True)
    print('\nFinal agent output:', out['answer'])
    print('History:')
    for h in out['history']:
        print(' ', h)
